# Module 0: Introduction

## Exercise 1: Setting Up Your Environment

In this exercise, you familiarize yourself with your equipment and verify that licenses are installed

**Objectives**

This exercise focuses on enabling you to do the following:
- Explore the Kubernetes cluster configuration
- Set up administrator access on the jumphost
- Configure the integrated development environment (IDE)
- Work with YAML files in the IDE

## Exercise Equipment

In this exercise, you use the following systems.

| System                  | Host Name   | IP Addresses   | User Name (case sensitive) | Password  |
|-------------------------|-------------|----------------|----------------------------|-----------|
| Linux Mint 20           | jumphost    | 192.168.0.5    | user                       | Netapp1!  |
| Kubernetes Control Plane| kubmas1-1   | 192.168.0.61   | root                       | Netapp1!  |
| Kubernetes Worker 1     | kubwor1-1   | 192.168.0.62   | root                       | Netapp1!  |
| Kubernetes Worker 2     | kubwor1-2   | 192.168.0.63   | root                       | Netapp1!  |
| Kubernetes Worker 3     | kubwor1-3   | 192.168.0.64   | root                       | Netapp1!  |

#### Task 3: Configure Kubernetes Administrator access on the jump host

In this task, you set up Kubernetes administrator access on the jump host.


---

Install kubectl by using the Linux Snap tool.


snap install kubectl --channel=1.29/stable --classic


---

In [3]:
snap install kubectl --channel=1.29/stable --classic


kubectl (1.29/stable) 1.29.15 from Canonical✓ installed  -.0ns


When requested to authenticate the installation of the package, enter Netapp1!


---

Click Authenticate to complete the installation.


---

Verify the installation and the version.


In [4]:
kubectl version --client -o json


{
  "clientVersion": {
    "major": "1",
    "minor": "29",
    "gitVersion": "v1.29.15",
    "gitCommit": "0d0f172cdf9fd42d6feee3467374b58d3e168df0",
    "gitTreeState": "clean",
    "buildDate": "2025-03-12T02:18:07Z",
    "goVersion": "go1.23.7",
    "compiler": "gc",
    "platform": "linux/amd64"
  },
  "kustomizeVersion": "v5.0.4-0.20230601165947-6ce0bf390ce3"
}


---

Verify the Kubernetes configuration.


In [5]:
kubectl config view


apiVersion: v1
clusters: null
contexts: null
current-context: ""
kind: Config
preferences: {}
users: null


---

The results are empty. 

You have not yet authenticated with the Kubernetes cluster on your jump host 

and you have no authorization to run any commands. 

You correct this situation now.


---

Run pwd to confirm that your working directory for the user account is /home/user.


In [3]:
pwd

/home/user/repos/STRSW-ILT-UATWK


---

Create a hidden folder called .kube under your home directory.


In [4]:
mkdir -p ~/.kube


---

Run the file [Exercise 0/exercise0Task4.sh](./Exercise%200/exercise0Task4.sh) to copy the kubeconfig from the control plane.

<details> <summary> Exercise 0/exercise0Task4.sh </summary>

[Exercise 0/exercise0Task4.sh](./Exercise%200/exercise0Task4.sh)

  ```bash
  # This script has been written for this exercise environment 
# and is not intented to be used in a production enviroment
# Execute by: ./exercise0Task4.sh
#!/bin/bash
DIR='/home/user/.kube' 
sshpass -p Netapp1! scp -o "StrictHostKeyChecking=no" root@kubmas1-1:/root/.kube/config config1 
sshpass -p Netapp1! scp -o "StrictHostKeyChecking=no" root@kubmas2-1:/root/.kube/config config2
sudo sed -i 's/\<kubernetes\>/source/g' config1 
sudo sed -i 's/\<kubernetes\>/destination/g' config2
konfig=$(KUBECONFIG=config1:config2 kubectl config view --flatten)
if [ ! -d "$DIR" ]; then
    mkdir -p $DIR;
fi
echo "$konfig" > $DIR/config
rm config1
rm config2
#echo "$konfig" > config
#$KUBECONFIG=$DIR/config


In [2]:
./Exercise\ 0/exercise0Task4.sh

In [3]:
kubectl config view

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: DATA+OMITTED
    server: https://192.168.0.96:6443
  name: destination
- cluster:
    certificate-authority-data: DATA+OMITTED
    server: https://192.168.0.61:6443
  name: source
contexts:
- context:
    cluster: destination
    user: destination-admin
  name: destination-admin@destination
- context:
    cluster: source
    user: source-admin
  name: source-admin@source
current-context: source-admin@source
kind: Config
preferences: {}
users:
- name: destination-admin
  user:
    client-certificate-data: DATA+OMITTED
    client-key-data: DATA+OMITTED
- name: source-admin
  user:
    client-certificate-data: DATA+OMITTED
    client-key-data: DATA+OMITTED


In November 2020, Docker Hub instituted a new policy that limits the
number of pulls available from an anonymous account, like the account that we use. 

If you
reach this limit, you see the following error in Events:

```
docker: Error response from daemon: toomanyrequests: You have reached
your pull rate limit. You may increase the limit by authenticating and
upgrading:
```   
https://www.docker.com/increase-rate-limit


Create a token variable:

```bash
TOKEN=$(curl "https://auth.docker.io/token?service=registry.docker.io&scope=repository:ratelimitpreview/test:pull" | jq -r .token)

In [4]:

TOKEN=$(curl "https://auth.docker.io/token?service=registry.docker.io&scope=repository:ratelimitpreview/test:pull" | jq -r .token)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5429    0  5429    0     0  29505      0 --:--:-- --:--:-- --:--:-- 29345



Use the token variable and review the HTML header output

In [5]:
curl --head -H "Authorization: Bearer $TOKEN" https://registry-1.docker.io/v2/ratelimitpreview/test/manifests/latest

HTTP/1.1 200 OK
content-length: 527
content-type: application/vnd.docker.distribution.manifest.v2+json
docker-content-digest: sha256:c2d41d2ba6d8b7b4a3ffec621578eb4d9a0909df29dfa2f6fd8a2e5fd0836aed
docker-distribution-api-version: registry/2.0
etag: "sha256:c2d41d2ba6d8b7b4a3ffec621578eb4d9a0909df29dfa2f6fd8a2e5fd0836aed"
date: Tue, 25 Mar 2025 18:46:34 GMT
strict-transport-security: max-age=31536000
ratelimit-limit: 100;w=21600
ratelimit-remaining: 26;w=21600
docker-ratelimit-source: 216.240.31.145



To resolve this restriction:

1. Navigate to Docker Hub (https://hub.docker.com) and register for a free account.

2. Use the authentication account when you request pulls from Kubernetes (you must
create the secret in every namespace in which you use the account):

    a. Create a secret with your authentication credentials:

    ```bash
      kubectl create secret docker-registry dockerhubkey \  
      --docker-username=[USERNAME] \  
      --docker-password=[PASSWORD] \  
      --docker-email=[EMAIL OF THE FREE ACCOUNT]  
    ```

    b. Edit the default service account and add the **imagePullSecrets** option:

    
      ```bash    
        kubectl -n default edit serviceaccount default  
      ```
      ```yaml
      apiVersion: v1  
      kind: ServiceAccount  
      metadata:  
        creationTimestamp: 2015-08-07T22:02:39Z  
        name: default  
        namespace: default  
        uid: 052fb0f4-3d50-11e5-b066-42010af0d7b6  
      secrets:  
      - name: default-token-uudge  
      imagePullSecrets:  
      - name: dockerhubkey  
      ```

Run the following commands in a terminal.  
You can open a terminal in VSCode using ```CTRL+` ```

In [7]:

kubectl create secret docker-registry dockerhubkey --docker-username=spatilsg --docker-password=Netapp.Lab.Mar2025 --docker-email=nihcas@gmail.com

secret/dockerhubkey created


In [8]:
kubectl patch serviceaccount default -p '{"imagePullSecrets": [{"name": "dockerhubkey"}]}'

serviceaccount/default patched


In [9]:
echo $TOKEN


eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsIng1YyI6WyJNSUlFRmpDQ0F2NmdBd0lCQWdJVUlvQW42a0k2MUs3bTAwNVhqcXVpKzRDTzVUb3dEUVlKS29aSWh2Y05BUUVMQlFBd2dZWXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJRXdwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSEV3bFFZV3h2SUVGc2RHOHhGVEFUQmdOVkJBb1RERVJ2WTJ0bGNpd2dTVzVqTGpFVU1CSUdBMVVFQ3hNTFJXNW5hVzVsWlhKcGJtY3hJVEFmQmdOVkJBTVRHRVJ2WTJ0bGNpd2dTVzVqTGlCRmJtY2dVbTl2ZENCRFFUQWVGdzB5TkRBNU1qUXlNalUxTURCYUZ3MHlOVEE1TWpReU1qVTFNREJhTUlHRk1Rc3dDUVlEVlFRR0V3SlZVekVUTUJFR0ExVUVDQk1LUTJGc2FXWnZjbTVwWVRFU01CQUdBMVVFQnhNSlVHRnNieUJCYkhSdk1SVXdFd1lEVlFRS0V3eEViMk5yWlhJc0lFbHVZeTR4RkRBU0JnTlZCQXNUQzBWdVoybHVaV1Z5YVc1bk1TQXdIZ1lEVlFRREV4ZEViMk5yWlhJc0lFbHVZeTRnUlc1bklFcFhWQ0JEUVRDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBTGRWRDVxNlJudkdETUxPVysrR1MxWENwR2FRRHd0V3FIS2tLYlM5cVlJMXdCallKWEJ6U2MweTBJK0swU0lVd2pqNGJJT3ZpNXNyOGhJajdReGhrY1ppTlU1OEE5NW5BeGVFS3lMaU9QU0tZK3Y5VnZadmNNT2NwVW1xZ1BxWkhoeTVuMW8xbGxmek92dTd5SDc4a1FyT0lTMTZ3RFVVZm8yRkxPaERDaElsbCtYa2VlbFB6c0tiRWo3ZGJqdXV6RGxIODlW

To verify the usage of a Docker account:

1. Install jQuery (if it is not already installed): sudo apt install -y jq  
2. Create a token variable, replacing username and password with your account details:  

```bash
MYTOKEN=$(curl --user 'username:password' "https://auth.docker.io/token?service=registry.docker.io&scope=repository:ratelimitpreview/test:pull" | jq -r .token)

```

3. Use the token variable and review the HTML header output:

```bash
curl --head -H "Authorization: Bearer $MYTOKEN" https://registry-1.docker.io/v2/ratelimitpreview/test/manifests/latest

```

4. Example output:  

    ```plain-text
    ratelimit-limit: 200  
    ratelimit-remaining: 176  
    ```

replace `username` and `password` in the code cell below to test if you are using your Docker Hub account

In [10]:
MYTOKEN=$(curl --user 'spatilsg:Netapp.Lab.Mar2025' "https://auth.docker.io/token?service=registry.docker.io&scope=repository:ratelimitpreview/test:pull" | jq -r .token)

curl --head -H "Authorization: Bearer $MYTOKEN" https://registry-1.docker.io/v2/ratelimitpreview/test/manifests/latest

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5705    0  5705    0     0  53820      0 --:--:-- --:--:-- --:--:-- 53820
HTTP/1.1 200 OK
content-length: 527
content-type: application/vnd.docker.distribution.manifest.v2+json
docker-content-digest: sha256:c2d41d2ba6d8b7b4a3ffec621578eb4d9a0909df29dfa2f6fd8a2e5fd0836aed
docker-distribution-api-version: registry/2.0
etag: "sha256:c2d41d2ba6d8b7b4a3ffec621578eb4d9a0909df29dfa2f6fd8a2e5fd0836aed"
date: Tue, 25 Mar 2025 18:55:58 GMT
strict-transport-security: max-age=31536000
ratelimit-limit: 200;w=21600
ratelimit-remaining: 200;w=21600
docker-ratelimit-source: b51dc179-fffd-4da5-8a20-f2521cc7b2e2

